# Problem Statement:

We want to train a simple feedforward neural network to calculate terminal velocity of a solid sphere of density $\sigma$ and radius r, in a liquid of density $\rho$ and viscosity $\eta$.

The formula for this terminal velocity classically is :
$$V_{t} = \frac{2r^{2}(\rho - \sigma)g}{9\eta}$$

Thus there will be 4 inputs into the neural network, densities $\sigma$ and $\rho$, viscosity $\eta$ and radius r. And there will be one output corresponding to the terminal velocity.

We will use above equation to simulate and create data to put into our neural network.


### The twist

We are only allowed to implement this neural network completely from scratch without any external libraries
Only python and numpy allowed (and Matplotlib for plotting purposes)

Additionally, I can only reference the 3b1b deep learning series for help, and chatgpt assisstance is limited to only conceptual doubts.

In [116]:
import numpy as np
import matplotlib.pyplot as plt

## Step 1: Creating the Data 

Chatgpt suggested these range of values for simulating the data

- > r from 10^-4 to 5 * 10^-3 m
- > $\sigma$ from 500 to 8000 kg/m^3
- > $\rho$ from 700 to 1300 kg/m^3
- > $\eta$ from 0.001 to 2 Pa $\cdot$ s

We will consider 5k input data points into the neural network

In [117]:
dataPoints = 5000

In [118]:
rng = np.random.default_rng(seed = 42)

r = rng.uniform(low = 0.0001, high = 0.005, size = (dataPoints, 1))
densSolid = rng.uniform(low = 500, high = 8000, size = (dataPoints, 1))
densLiquid = rng.uniform(low = 700, high = 1300, size = (dataPoints, 1))
viscosity = rng.uniform(low = 0.001, high = 2, size = (dataPoints, 1))

data = np.hstack((r, densSolid, densLiquid, viscosity))

In [119]:
def terminal(dats):
    r, densL, densS, visc = dats
    return (2*(r**2)*(densL - densS)*9.8)/(9*visc)

v = np.apply_along_axis(terminal, 1, data)

## Step 2: Train/test split and Scaling the data

We shall do a 80/20 standard split

We will use Z-score scaling or standardization to scale
We will learn the means and stds from training data and apply same on testing data

We will scale both inputs and outputs since thats necessary for neural networks, and we will unscale the outputs at the end

In [120]:
index = int((0.8) * data.shape[0])

xTrain = data[:index, :]
xTest = data[index: , :]
yTrain = v[:index]
yTest = v[index:]

In [121]:
#Scaling inputs
xMean = np.mean(xTrain, axis = 0)[np.newaxis, :]
xStd = np.std(xTrain, axis = 0)[np.newaxis, :]

xTrain = (xTrain - xMean)/xStd
xTest = (xTest - xMean)/xStd

#Scaling outputs
yMean = np.mean(yTrain)
yStd = np.std(yTrain)
yTestOrig = yTest
yTrainOrig = yTrain

yTrain = (yTrain - yMean)/yStd
yTest = (yTest - yMean)/yStd

In [122]:
yScalingParams = np.array([yMean, yStd])

np.save("./data/xTrain.npy", xTrain)
np.save("./data/xTest.npy", xTest)
np.save("./data/yTrain.npy", yTrain)
np.save("./data/yTest.npy", yTest)
np.save("./data/yTestOrig.npy", yTestOrig)
np.save("./data/yTrainOrig.npy", yTrainOrig)
np.save("./data/yScalingParams.npy", yScalingParams)

## Step 3: Create Base Neural Network Architecture

Each layer requires knowledge of previous layers, as thats how number of weights are decided

The way we will construct this is using 2 classes, one for each layer, and another for the entire model on its own 

Each layer will take number of neurons from previous layer and construct its own weight and bias matrices, randomly of correct size, and we will create a forward function as well

The activation function we will use in this neural network is the ReLU (Rectified Linear Unit)

Important thing to note is that we shouldn't have an activation for our output layer, since our final output (terminal velocity) can and should take any value and shouldn't only be restricted to +ve values (which ReLU would do)

Let's initially consider an architecture where inputs -> layer 1 -> layer 2 -> output

Hidden layer 1 has 12 neurons, and hidden layer 2 has 8 neurons

In [123]:
class Layer:
    #Consider number of neurons in previous layer as x and number of layers in current layer as n
    def __init__(self, x, n):
        self.neurons = n
        
        self.weights = rng.uniform(low = -0.3, high = 0.3, size = (n, x))
        self.biases = rng.uniform(low = -0.1, high = 0.1, size = (n, 1))
    
    def ReLU(self, z):
        return np.maximum(0, z)

    #a is the matrix corresponding to the activations of previous layer
    def forward(self, a, hidden):
        return self.ReLU(np.matmul(self.weights, a) + self.biases) if hidden else np.matmul(self.weights, a) + self.biases

The loss function we will be using is the MSE (Mean Squared Error) loss

In [124]:
l1Neurons = 12
l2Neurons = 8

class NN:
    def __init__(self, Train, Test):
        self.l1 = Layer(4, l1Neurons)
        self.l2 = Layer(self.l1.neurons, l2Neurons)
        self.output = Layer(self.l2.neurons, 1)

        self.train = Train.T
        self.test = Test.T
    
    def forwardpass(self, input):
        a1 = self.l1.forward(input, 1)
        a2 = self.l2.forward(a1, 1)
        output = self.output.forward(a2, 0)
        return output

    def loss(self, pred, actual):
        return np.mean(((actual - pred) ** 2))
    
    def testing(self):
        loss = self.loss(self.forwardpass(self.train), self.test)
        print(f"The mean loss of our model is {loss}")
        return loss

### Experiment 1: 

We have implemented our basic framework to allow for forwardpass, so let's try our forwardpass on completely random values to set a baseline and to test our code thus far

In [125]:
nn1 = NN(xTrain, yTrain)
loss = nn1.testing()

The mean loss of our model is 1.0090402535882859
